In [1]:
from typing_extensions import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from typing_extensions import TypedDict
import os
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from IPython.display import Markdown

In [2]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq


llm=ChatGroq(model="qwen-2.5-32b")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("Hello")
result

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_time': 0.05, 'prompt_time': 0.003732836, 'queue_time': 0.051870863, 'total_time': 0.053732836}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-102cc314-6ed9-4171-b88f-14d308803e06-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [3]:
# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["Bullish", "Bearish", "Sideways"] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


In [4]:
# State
class State(TypedDict):
    input: str
    decision: str
    output: str
    input_BEST : str


In [5]:
# Nodes
def llm_call_1(state: State):
    """Suggest Futures and Options stratergy for a Bullish Market"""

    result = llm.invoke("Futures and Options stratergy for" + state["input_BEST"])
    return {"output": result.content}


def llm_call_2(state: State):
    """Suggest Futures and Options stratergy for a Bearish Market"""

    print("LLM call 2 is called")

    result = llm.invoke("Futures and Options stratergy for" + state["input_BEST"])
    return {"output": result.content}


def llm_call_3(state: State):
    """Suggest Futures and Options stratergy for a Sideways Market"""

    result = llm.invoke("Futures and Options stratergy for" + state["input_BEST"])
    return {"output": result.content}

In [6]:
def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to Bullish, Bearish, or Sideways based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}

In [7]:
# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "Bullish":
        return "llm_call_1"
    elif state["decision"] == "Bearish":
        return "llm_call_2"
    elif state["decision"] == "Sideways":
        return "llm_call_3"

In [8]:
def llm_call_1_best(state: State):
    """Score Futures and Options stratergy for a  Market based on row risk and high reward"""

    result = llm.invoke("Futures and Options stratergy for" + state["input"])
    return {"input_BEST": result.content}

def llm_call_2_best(state: State):
    """Score Futures and Options stratergy for a Bearish Market based on row risk and high reward"""

    result = llm.invoke("Futures and Options stratergy for" + state["output"])
    return {"output_BEST": result.content}

def llm_call_3_best(state: State):
    """Score Futures and Options stratergy for a Sideways Market based on row risk and high reward"""

    result = llm.invoke("Futures and Options stratergy for" + state["output"])
    return {"output_BEST": result.content}

In [9]:
# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "Bullish":
        return "llm_call_1"
    elif state["decision"] == "Bearish":
        return "llm_call_2"
    elif state["decision"] == "Sideways":
        return "llm_call_3"

In [13]:
# Build workflow
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)
router_builder.add_node("llm_call_1_best", llm_call_1_best)
router_builder.add_node("llm_call_2_best", llm_call_2_best)
router_builder.add_node("llm_call_3_best", llm_call_3_best)
# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_1_best")
router_builder.add_edge(START, "llm_call_2_best")
router_builder.add_edge(START, "llm_call_3_best")
router_builder.add_edge("llm_call_1_best", "llm_call_1")
router_builder.add_edge("llm_call_2_best", "llm_call_2")
router_builder.add_edge("llm_call_3_best", "llm_call_3")
#router_builder.add_edge("llm_call_1_best", "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_1","llm_call_2","llm_call_3",
    route_decision,
    {  # Name returned by route_decision : Name of next node to visit
        "llm_call_1": END,
        "llm_call_2": END,
        "llm_call_3": END,
    },
)
#router_builder.add_edge("llm_call_1","llm_call_1_best" )
#router_builder.add_edge("llm_call_2","llm_call_2_best")
#router_builder.add_edge("llm_call_3", END)
#router_builder.add_edge("llm_call_1_best", END)
#router_builder.add_edge("llm_call_2_best", END)
#router_builder.add_edge("llm_call_3_best", END)

# Compile workflow
router_workflow = router_builder.compile()

# Show the workflow
display(Image(router_workflow.get_graph().draw_mermaid_png()))

TypeError: Graph.add_conditional_edges() takes from 3 to 5 positional arguments but 6 were given

In [11]:
# Invoke
state = router_workflow.invoke({"input": "Suggest a stratergy for a bullish market"})
Markdown(state["output"])

KeyError: 'output'